# mdb_to_df

In [1]:
import pyodbc
import os
import pandas as pd
from zipfile import ZipFile


def mdb_to_df(file_name, sql):

    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        fr'DBQ={file_name};'
    )
    print(conn_str)
    cnxn = pyodbc.connect(conn_str)

    df = pd.read_sql(sql, cnxn)

    print(f'{file_name} Loaded')
    return df

# Read SUM

In [2]:
import numpy as np
from datetime import datetime as dt


def read_sum(period):
    usecols_sum = """
    SELECT CDbl(TimeOn) AS TOn, CDbl(TimeOff) AS TOff,
    StationNr, Alarmcode, ID, Parameter
    FROM tblAlarmLog WHERE TimeOff IS NOT NULL
    union
    SELECT CDbl(TimeOn) AS TOn, TimeOff AS TOff,
    StationNr, Alarmcode, ID, Parameter
    FROM tblAlarmLog WHERE TimeOff IS NULL
    """
    file_name = f'../DATA/SUM/{period}-sum.mdb'
    alarms = mdb_to_df(file_name=file_name, sql=usecols_sum)

    alarms['TOn'] = sqldate_to_datetime(alarms['TOn'])
    alarms['TOff'] = sqldate_to_datetime(alarms['TOff'])

    alarms.rename(columns={'TOn': 'TimeOn',
                           'TOff': 'TimeOff'}, inplace=True)

    alarms = alarms[alarms.StationNr >= 2307405]

    alarms = alarms[
        alarms.StationNr <= 2307535].reset_index(
        drop=True)

    alarms.dropna(subset=['Alarmcode'], inplace=True)

    alarms.reset_index(drop=True, inplace=True)

    alarms.Alarmcode = alarms.Alarmcode.astype(int)

    return alarms


def sqldate_to_datetime(column):
    try:
        column = column.str.replace(',', '.').astype(float)
    except:
        pass
    day_parts = np.modf(column.loc[~column.isna()])

    column.loc[~column.isna()] = (
        dt(1899, 12, 30) +
        day_parts[1].astype('timedelta64[D]', errors='ignore') +
        (day_parts[0] * 86400000).astype('timedelta64[ms]', errors='ignore')
    )
    column = column.fillna(pd.NaT)
    return column

# Cascade

In [3]:
# Determine alarms real periods
def cascade(df):

    df.reset_index(inplace=True, drop=True)
    df['TimeOffMax'] = df.TimeOff.cummax().shift()

    df.at[0, 'TimeOffMax'] = df.at[0, 'TimeOn']

    return df


# looping through turbines and applying cascade method
def apply_cascade(result_sum):

    # Sort by alarm ID
    result_sum.sort_values(['ID'], inplace=True)
    df = result_sum.groupby('StationId').apply(cascade)

    mask_root = (df.TimeOn.values >= df.TimeOffMax.values)
    mask_children = (df.TimeOn.values < df.TimeOffMax.values) & (
        df.TimeOff.values > df.TimeOffMax.values)
    mask_embedded = (df.TimeOff.values <= df.TimeOffMax.values)

    df.loc[mask_root, 'NewTimeOn'] = df.loc[mask_root, 'TimeOn']
    df.loc[mask_children, 'NewTimeOn'] = df.loc[mask_children, 'TimeOffMax']
    df.loc[mask_embedded, 'NewTimeOn'] = df.loc[mask_embedded, 'TimeOff']

    df.drop(columns=['TimeOffMax'], inplace=True)

    df.reset_index(inplace=True, drop=True)

    TimeOff = df.TimeOff
    NewTimeOn = df.NewTimeOn

    df['RealPeriod'] = abs(TimeOff - NewTimeOn)

    mask_siemens = (df['Error Type'] == 1)
    mask_tarec = (df['Error Type'] == 0)

    df['Period Siemens(s)'] = df[mask_siemens].RealPeriod  # .dt.seconds
    df['Period Tarec(s)'] = df[mask_tarec].RealPeriod  # .dt.seconds
    # df['RealPeriod'] = df['RealPeriod'].dt.seconds

    return df

# Periods

In [4]:
from dateutil.relativedelta import relativedelta

period = input('period ex: "2020-07"') or '2020-07'

period_dt = dt.strptime(period, "%Y-%m")
period_month = period_dt.month
period_year = period_dt.year

days_in_period = pd.Period(f'{period}').days_in_month

previous_period_dt = period_dt + relativedelta(months=-1)
previous_period = previous_period_dt.strftime("%Y-%m")

lastday_period_dt = period_dt + relativedelta(months=1, days=-1)
lastday_period = lastday_period_dt.strftime("%Y-%m-%d")


lastday_previous_period_dt = period_dt + relativedelta(days=-1)
lastday_previous_period = lastday_previous_period_dt.strftime("%Y-%m-%d")

days_to_previous_period = len(pd.date_range(
    f'{period_year}-01-01', lastday_previous_period_dt))

# Import last period cumul

In [5]:
df_last_ax1 = pd.read_excel(f'./input/{previous_period}/output_xlsxwriter.xlsx', sheet_name= 'ax1', index_col=0)


last_boost_cumul = df_last_ax1.iat[0, 7]
last_ratio_cumul = df_last_ax1.iat[0, 8]
last_mtbf_cumul = df_last_ax1.iat[0, 9]
last_mttr_cumul = df_last_ax1.iat[0, 10]
last_mtti_cumul = df_last_ax1.iat[0, 11]
last_onee_cumul = df_last_ax1.iat[0, 12]

IndexError: index 12 is out of bounds for axis 0 with size 12

In [6]:
df_last_ax1

,Indispo. Total %,Indispo. Tarec %,Indispo. Siemens %,Indispo. ONEE %,Indispo. Ebop %,Pertes élctriques \n en MWh,Power Boost en MWh,Performance moyenne \n des turbines,MTBF - Mean Time \n Between Failure,MTTR - Mean Time \n To Repair,MTTI - Mean Time \n To Intervention,Compteurs ONEE MWh
LTA-Lost Time,,,,,,,,,,,,
NaN,NaN,NaN,NaN,NaN,NaN,11138.13,3458.1,94.846,37.718,11.2,8.122,554999.62


# ONEE

In [7]:
onee_period = float(input('onee_period') or 167482.8)

onee_cumul = (last_onee_cumul + onee_period)

NameError: name 'last_onee_cumul' is not defined

# Power Boost

In [8]:

def read_boost(file_name):
    usecols_cnt = '''TimeStamp, StationId, wtc_BoostKWh_endvalue'''

    sql_cnt = f"Select {usecols_cnt} FROM tblSCTurCount;"

    cnt = mdb_to_df(file_name=file_name, sql=sql_cnt)

    cnt['TimeStamp'] = pd.to_datetime(
        cnt['TimeStamp'], format='%m/%d/%y %H:%M:%S')

    return cnt

def boost_endvalue_check(df):
    df = df.reset_index().sort_values('TimeStamp')
    df.drop('index', axis=1, inplace=True)
    clmn = df['wtc_BoostKWh_endvalue']
    x = (clmn.iat[-1] - clmn.iat[0])/1000
    # df = df.diff().query('wtc_BoostKWh_endvalue < 0')
    if x < 0:
        return 0
    else:
        return x



In [9]:

# last_boost_cumul = input('last_boost_cumul') or 3458.1

boost = read_boost(f'../Availability_Warranty_Dash/monthly_data/uploads/{period}/{period}-cnt.mdb')

boost_period = boost.groupby('StationId').apply(lambda df: boost_endvalue_check(df)).sum()

boost_cumul = last_boost_cumul + boost_period

DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../Availability_Warranty_Dash/monthly_data/uploads/2020-07/2020-07-cnt.mdb;


Error: ('HY000', "[HY000] [Microsoft][ODBC Microsoft Access Driver]General error Unable to open registry key Temporary (volatile) Ace DSN for process 0x30b4 Thread 0x25d0 DBC 0x25d900b8                                                              Jet'. (63) (SQLDriverConnect); [HY000] [Microsoft][ODBC Microsoft Access Driver]General error Unable to open registry key Temporary (volatile) Ace DSN for process 0x30b4 Thread 0x25d0 DBC 0x25d900b8                                                              Jet'. (63); [HY000] [Microsoft][ODBC Microsoft Access Driver] Could not find file '(unknown)'. (-1811); [HY000] [Microsoft][ODBC Microsoft Access Driver]General error Unable to open registry key Temporary (volatile) Ace DSN for process 0x30b4 Thread 0x25d0 DBC 0x25d900b8                                                              Jet'. (63); [HY000] [Microsoft][ODBC Microsoft Access Driver]General error Unable to open registry key Temporary (volatile) Ace DSN for process 0x30b4 Thread 0x

In [10]:
boost_period, boost_cumul

NameError: name 'boost_period' is not defined

# Input MTBF MTTR MTTI 

In [11]:
MTTI = pd.read_html(f'./input/{period}/Analyse_ID-173_MTTI.html', header=0)[0]
MTTI

,StationNr,StationCount,MTTI en (s),Nombre de valeurs,Somme des valeurs,Nombre d'erreurs
0,2307405,1.0,NaN,NaN,NaN,NaN
1,2307406,2.0,49614.0,1.0,49614.0,0.0
2,2307407,3.0,NaN,NaN,NaN,NaN
3,2307408,4.0,60897.0,1.0,60897.0,0.0
4,2307409,5.0,34306.0,2.0,68612.0,0.0
...,...,...,...,...,...,...
127,2307532,128.0,19.0,1.0,19.0,0.0
128,2307533,129.0,NaN,NaN,NaN,NaN
129,2307534,130.0,NaN,NaN,NaN,NaN
130,2307535,131.0,39606.0,2.0,79212.0,0.0


In [12]:
mtti_period = MTTI.iat[-1,2]/3600
mtti_period

7.875

In [13]:
# MTTI cumulé
mtti_cumul = (last_mtti_cumul*days_to_previous_period + mtti_period *
              days_in_period)/(days_to_previous_period+days_in_period)

f'({last_mtti_cumul}*{days_to_previous_period} + {mtti_period}*{days_in_period})/({days_to_previous_period}+{days_in_period})'

'(554999.62*182 + 7.875*31)/(182+31)'

In [14]:
MTBF_MTTR_cat_prev_cumul = pd.read_excel(
    f'./input/{previous_period}/output_xlsxwriter.xlsx', sheet_name='MTBF & MTTR cumul')

MTBF_MTTR_cat_prev_cumul.dropna(
    how='all', subset=['MTBF en j', 'MTTR en h'], inplace=True)

MTBF_MTTR_cat_prev_cumul.rename(
    columns={'Catégorie Alarme': 'Catégorie'}, inplace=True)

MTBF_MTTR_cat_prev_cumul

,Catégorie,Durée Alarme,Fréquence Alarme,MTTR en h,MTBF en j
0,System,1727.498056,185,9.337827,128.486600
1,Generator,307.007778,9,34.111975,2647.689779
2,Hub,454.496944,37,12.283701,643.866557
3,Gear,548.919167,45,12.198204,529.313964
4,Grid,1426.620833,161,8.860999,147.717748
5,Rotor,154.212778,18,8.567377,1324.198582
6,Hydraulics,1939.796667,186,10.429014,127.748253
7,Environment,124.631667,21,5.934841,1135.086048
9,Brake,350.368333,35,10.010524,680.782895
10,Yaw,2863.508333,252,11.363128,94.137647


In [15]:
MTBF_MTTR_cat = pd.read_excel(f'./input/{period}/MTBF & MTTR.xlsx')

MTBF_MTTR_GLOBAL, MTBF_MTTR_cat = MTBF_MTTR_cat.iloc[0], MTBF_MTTR_cat.iloc[1:]

MTBF_MTTR_cat.dropna(how='all', subset=['MTBF', 'MTTR'], inplace=True)

In [16]:
MTBF_MTTR_GLOBAL

Catégorie                    <GLOBAL>
MTBF                        4.188e+06
MTBF (Temps total)         3.5087e+08
MTBF (Temps d'arrêt)      3.26645e+06
MTBF (Nombre d'arrêts)             83
MTTR                          39354.8
MTTR (Temps d'arrêt)      3.26645e+06
MTTR (Nombre d'arrêts)             83
Name: 0, dtype: object

In [17]:
MTBF_MTTR_cat_cumul = MTBF_MTTR_cat.merge(MTBF_MTTR_cat_prev_cumul, on=['Catégorie'], how='outer').fillna(0)
MTBF_MTTR_cat_cumul

,Catégorie,MTBF,MTBF (Temps total),MTBF (Temps d'arrêt),MTBF (Nombre d'arrêts),MTTR,MTTR (Temps d'arrêt),MTTR (Nombre d'arrêts),Durée Alarme,Fréquence Alarme,MTTR en h,MTBF en j
0,Brake,8.767718e+07,350870400.0,161696.0,4.0,40424.0000,161696.0,4.0,350.368333,35,10.010524,680.782895
1,Controller,4.380744e+07,350870400.0,410892.0,8.0,51361.5000,410892.0,8.0,1771.196389,152,11.652608,156.369738
2,Converter,1.750276e+07,350870400.0,815134.0,20.0,40756.7000,815134.0,20.0,1157.399167,95,12.183149,250.460790
3,Gear,3.896020e+07,350870400.0,228639.0,9.0,25404.3333,228639.0,9.0,548.919167,45,12.198204,529.313964
4,Grid,2.919987e+07,350870400.0,471961.0,12.0,39330.0833,471961.0,12.0,1426.620833,161,8.860999,147.717748
5,Hub,5.009759e+07,350870400.0,187270.0,7.0,26752.8571,187270.0,7.0,454.496944,37,12.283701,643.866557
6,Hydraulics,3.896579e+07,350870400.0,178324.0,9.0,19813.7777,178324.0,9.0,1939.796667,186,10.429014,127.748253
7,Miscellaneous,3.507746e+08,350870400.0,95773.0,1.0,95773.0000,95773.0,1.0,3.450000,1,3.450000,23841.856250
8,Rotor,1.754211e+08,350870400.0,28157.0,2.0,14078.5000,28157.0,2.0,154.212778,18,8.567377,1324.198582
9,System,2.335240e+07,350870400.0,584336.0,15.0,38955.7333,584336.0,15.0,1727.498056,185,9.337827,128.486600


In [18]:
# Generate the new calculated MTTBF and MTTR for each error group

temps_total_cumul = len(pd.date_range(
    f'{period_year}-01-01', f'{lastday_period}', freq='D'))*24*131

duree_cumul = MTBF_MTTR_cat_cumul['Durée Alarme'] + \
    MTBF_MTTR_cat_cumul['MTBF (Temps d\'arrêt)']/3600

MTBF_MTTR_cat_cumul['Durée Alarme'] = duree_cumul

freq_cumul = MTBF_MTTR_cat_cumul['MTBF (Nombre d\'arrêts)'] + \
    MTBF_MTTR_cat_cumul['Fréquence Alarme']

MTBF_MTTR_cat_cumul['Fréquence Alarme'] = freq_cumul

MTBF_MTTR_cat_cumul['MTBF en j'] = (
    temps_total_cumul - duree_cumul) / (freq_cumul * 24)

MTBF_MTTR_cat_cumul['MTTR en h'] = duree_cumul/freq_cumul

In [19]:
MTBF_MTTR_cat_cumul

,Catégorie,MTBF,MTBF (Temps total),MTBF (Temps d'arrêt),MTBF (Nombre d'arrêts),MTTR,MTTR (Temps d'arrêt),MTTR (Nombre d'arrêts),Durée Alarme,Fréquence Alarme,MTTR en h,MTBF en j
0,Brake,8.767718e+07,350870400.0,161696.0,4.0,40424.0000,161696.0,4.0,395.283889,39.0,10.135484,715.039227
1,Controller,4.380744e+07,350870400.0,410892.0,8.0,51361.5000,410892.0,8.0,1885.333056,160.0,11.783332,173.902778
2,Converter,1.750276e+07,350870400.0,815134.0,20.0,40756.7000,815134.0,20.0,1383.825278,115.0,12.033263,242.133397
3,Gear,3.896020e+07,350870400.0,228639.0,9.0,25404.3333,228639.0,9.0,612.430000,54.0,11.341296,516.249668
4,Grid,2.919987e+07,350870400.0,471961.0,12.0,39330.0833,471961.0,12.0,1557.721111,173.0,9.004168,160.913844
5,Hub,5.009759e+07,350870400.0,187270.0,7.0,26752.8571,187270.0,7.0,506.516389,44.0,11.511736,633.679435
6,Hydraulics,3.896579e+07,350870400.0,178324.0,9.0,19813.7777,178324.0,9.0,1989.331111,195.0,10.201698,142.667237
7,Miscellaneous,3.507746e+08,350870400.0,95773.0,1.0,95773.0000,95773.0,1.0,30.053611,2.0,15.026806,13950.873883
8,Rotor,1.754211e+08,350870400.0,28157.0,2.0,14078.5000,28157.0,2.0,162.034167,20.0,8.101708,1394.812429
9,System,2.335240e+07,350870400.0,584336.0,15.0,38955.7333,584336.0,15.0,1889.813611,200.0,9.449068,139.121289


In [19]:
df_ax4 =  MTBF_MTTR_cat_cumul[['Catégorie', 'MTTR en h', 'MTBF en j']]

df_ax4.rename(columns={'MTTR en h': 'MTTR', 'MTBF en j': 'MTBF'}, inplace=True)

In [20]:
MTBF_MTTR_cat_current_cumul = MTBF_MTTR_cat_cumul[[
    'Catégorie', 'Durée Alarme', 'Fréquence Alarme', 'MTTR en h', 'MTBF en j']]

MTBF_MTTR_cat_current_cumul.to_excel(
    f'./input/{period}/MTBF & MTTR cumul.xlsx')

In [21]:
duree_period = MTBF_MTTR_GLOBAL['MTBF (Temps d\'arrêt)']/3600

freq_period = MTBF_MTTR_GLOBAL['MTBF (Nombre d\'arrêts)']

mtbf_period = (days_in_period*24*131 - duree_period) / freq_period /24
mttr_period = duree_period/freq_period

In [22]:
mtbf_cumul = (last_mtbf_cumul*days_to_previous_period + mtbf_period *
              days_in_period)/(days_to_previous_period+days_in_period)

mttr_cumul = (last_mttr_cumul*days_to_previous_period + mttr_period *
              days_in_period)/(days_to_previous_period+days_in_period)

(mtbf_cumul, mttr_cumul)

(39.283167497978354, 11.160979426312448)

In [ ]:
#

# df ax 19 Performance ratio 

In [23]:

df_ax19 = pd.read_html(f'./input/{period}/TAREC REPORTING.html', thousands=' ', decimal=',')[0]

df_ax19['Turbine'] = df_ax19['Turbine'].str.extract(r'\[[^\d]*(\d+)[^\d]*\]').values.astype(int)

df_ax19 = df_ax19[['Turbine', 'Ratio']]

df_ax19.Ratio.mean()

# last_ratio_cumul = input('last_ performance_ratio_cumul') or 94.846

ratio_period = df_ax19.Ratio.mean()

ratio_cumul = (last_ratio_cumul*days_to_previous_period + ratio_period*days_in_period)/(days_to_previous_period+days_in_period)

In [54]:
df_ax19

,Turbine,Ratio
0,1,110.80
1,2,110.85
2,3,110.49
3,4,110.58
4,5,110.39
...,...,...
126,127,102.23
127,128,101.84
128,129,101.20
129,130,102.00


# Calcul mois

In [24]:
import pandas as pd

df = pd.read_csv(f'../Availability_Warranty_Dash/monthly_data/results/{period}-Availability.csv',
                    decimal=',', sep=';')
df.columns = ['_'.join(str(v) for v in tup) if type(tup) is tuple else tup for tup in df.columns]
df.TimeStamp = pd.to_datetime(df.TimeStamp)
df.to_pickle(f'../Data/results/{period}.pkl')

In [25]:


alarms = read_sum(period)

results = pd.read_pickle(f'../DATA/results/{period}.pkl')

alarms.rename(columns={'StationNr': 'StationId'}, inplace=True)

alarms['StationId'] = alarms['StationId'] - 2307404
results['StationId'] = results['StationId'] - 2307404



DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2020-08-sum.mdb;
../DATA/SUM/2020-08-sum.mdb Loaded


C:\Users\Saad\Miniconda3\envs\dash-env\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [26]:
# abbreviations
Frame = pd.DataFrame
c1 = 'Alarmcode'
c2 = 'Error Group'

error_group = pd.concat([Frame({c1: range(901, 2101), c2: 'System'}),
                         Frame({c1: range(2101, 3000), c2: 'Generator'}),
                         Frame({c1: range(3100, 4000), c2: 'Hub'}),
                         Frame({c1: range(4100, 5000), c2: 'Gear'}),
                         Frame({c1: range(5000, 6000), c2: 'Grid'}),
                         Frame({c1: range(6100, 7000), c2: 'Rotor'}),
                         Frame({c1: range(7100, 8000), c2: 'Hydraulics'}),
                         Frame({c1: range(8000, 8400), c2: 'Environement'}),
                         Frame({c1: range(8450, 9000),
                                c2: 'Turbine cond...'}),
                         Frame({c1: range(9100, 10000), c2: 'Brake'}),
                         Frame({c1: range(10100, 11000), c2: 'Yaw'}),
                         Frame({c1: range(11100, 12000), c2: 'PFC'}),
                         Frame({c1: range(12100, 13000), c2: 'Transformer'}),
                         Frame({c1: range(13000, 14000), c2: 'Converter-1'}),
                         Frame({c1: range(14000, 15000), c2: 'Gen.inverter'}),
                         Frame({c1: range(15000, 16000), c2: 'Grid inverter'}),
                         Frame({c1: range(16000, 17000), c2: 'Main bearing'}),
                         Frame({c1: range(17000, 18300), c2: 'Converter-2'}),
                         Frame({c1: range(62001, 64000), c2: 'Controller'}),
                         Frame({c1: range(64000, 65200), c2: 'MISCELLANEOUS'})])


In [27]:
reorder = ['System',
           'Generator',
           'Hub',
           'Gear',
           'Grid',
           'Rotor',
           'Hydraulics',
           'Environement',
           'Turbine cond...',
           'Brake',
           'Yaw',
           'PFC',
           'Transformer',
           'Converter-1',
           'Gen.inverter',
           'Grid inverter',
           'Main bearing',
           'Converter-2',
           'Controller',
           'MISCELLANEOUS']

In [28]:
error_list = pd.read_excel(
    r'Error_Type_List_Las_Update_151209.xlsx',
    usecols=lambda x: x != 'Type Selected')

error_list.Alarmcode = error_list.Alarmcode.astype(int)  # ,errors='ignore'

error_list.drop_duplicates(subset=['Alarmcode'], inplace=True)

error_list = error_list.merge(error_group, on='Alarmcode', how='left')

# ------------------------------------------------------------------------
''' label scada alarms with coresponding error type
and only keep alarm codes in error list'''
result_sum = pd.merge(alarms, error_list,
                        on='Alarmcode',
                        how='inner', sort=False)

# Remove warnings
result_sum = result_sum.loc[result_sum['Error Type'].isin([1, 0])]

# apply cascade
alarms_result_sum = apply_cascade(result_sum)

# only keep  parent alarms
parent_result_sum = alarms_result_sum.query('TimeOn == NewTimeOn')

# dash duree
main_result_sum = alarms_result_sum.query('RealPeriod > @pd.Timedelta(0)')
# main_result_sum['ALL duree'] = main_result_sum['TimeOff'] -main_result_sum['TimeOn']

In [29]:
alarms

,TimeOn,TimeOff,StationId,Alarmcode,ID,Parameter
0,2020-08-01 00:00:34.040,NaT,85,67,3088903,3.0 ...
1,2020-08-01 00:00:34.050,NaT,85,68,3088904,478 ...
2,2020-08-01 00:00:34.059,NaT,85,69,3088905,21.0 ...
3,2020-08-01 00:00:34.069,NaT,85,61,3088906,Out Of Range ...
4,2020-08-01 00:01:15.779,2020-08-01 00:20:12.710,4,50950,3088907,...
...,...,...,...,...,...,...
74778,2020-08-31 23:57:42.300,2020-08-31 23:59:57.869,85,50950,3164889,...
74779,2020-08-31 23:58:32.000,2020-09-01 02:02:02.000,15,63004,3167101,...
74780,2020-08-31 23:58:34.000,2020-09-01 01:59:27.000,15,63003,3167102,...
74781,2020-08-31 23:58:34.060,NaT,15,9,3167105,0 ...


## Graphe 3 ax5

In [30]:
df_ax5 = (main_result_sum.groupby('Error Group')
                  .agg(Freq=('Alarmcode', 'count'),
                       Durée=('RealPeriod', lambda x: x.sum().total_seconds()/3600))
                  .reindex(reorder)
                  .dropna()
                  .reset_index()
                  )

# df_ax5.plot(kind='bar', x='Error Group', ax=ax5)
df_ax5

,Error Group,Freq,Durée
0,System,406.0,538.132224
1,Generator,17.0,14.600000
2,Hub,649.0,165.998608
3,Gear,51.0,149.658612
4,Grid,172.0,229.364172
5,Rotor,14.0,20.085279
6,Hydraulics,42.0,51.840553
7,Environement,22.0,21.005278
8,Brake,36.0,27.922779
9,Yaw,246.0,274.385002


## Graphe 4 ax6

In [31]:
df_ax6 = (main_result_sum.groupby('Alarmcode')
                .agg(Freq=('Alarmcode', 'count'),
                     Durée=('RealPeriod',
                               lambda x: x.sum().total_seconds()/3600))
                .sort_values('Durée', ascending=False)
                .head(20)
                .reset_index()
                .sort_values('Durée', ascending=False))

# df_ax6.plot(kind='bar', x='Alarmcode', ax=ax6)
df_ax6

,Alarmcode,Freq,Durée
0,1001,225,445.754999
1,5104,87,211.717506
2,10100,51,173.670279
3,13902,178,129.113865
4,4104,18,94.388056
5,10109,51,71.231666
6,3145,11,57.336666
7,64066,8,43.089445
8,1018,100,37.664168
9,3130,579,37.422220


## ax7

In [32]:

most_categories = main_result_sum.groupby('Error Group').agg({'RealPeriod': np.sum}).sort_values('RealPeriod').tail(4).index.values
df_most_categories = main_result_sum.loc[main_result_sum['Error Group'].isin(most_categories)]
df_most_categories


# ------------------------------------------------------------------------------------------------
def extract_important_alarms(df):

    df = df.groupby('Alarmcode').agg({'RealPeriod': np.sum}).reset_index()
    df['percentage'] = 100 * df['RealPeriod'] / df['RealPeriod'].sum()
    df.sort_values('percentage', ascending=False, inplace=True)
    df['percentage roll'] = df['percentage'].cumsum()

    for i in range(1,100):
        if len(df.query('`percentage roll` < @i')) < 3:
            continue
        else:
            df = df.query('`percentage roll` < @i')
        break

    return df

# ------------------------------------------------------------------------------------------------

df = df_most_categories.query('`Error Group` == "Converter-1"')

df = df.groupby('Alarmcode').agg({'RealPeriod': np.sum}).reset_index()
df['percentage'] = 100 * df['RealPeriod'] / df['RealPeriod'].sum()
df.sort_values('percentage', ascending=False, inplace=True)
df.reset_index(inplace=True, drop=True)
df['percentage roll'] = df['percentage'].cumsum()
df

# ------------------------------------------------------------------------------------------------

df_ax7 = (df_most_categories.groupby('Error Group')
                             .apply(lambda x: extract_important_alarms(x))
                             .reset_index()
                             .merge(error_list[['UK Text', 'Alarmcode']]))

df_ax7 = (df_most_categories.groupby('Error Group')
                             .apply(lambda x: (x.groupby('Alarmcode')
                                                .agg({'RealPeriod': np.sum})
                                                .sort_values('RealPeriod')
                                                .tail(4)))
                             .reset_index()
                             .merge(error_list[['UK Text', 'Alarmcode']]))

df_ax7
# ------------------------------------------------------------------------------------------------
ax_7 = '''

Les alarmes "System" les plus importantes sont 
    - "1001 - Manual Stop"  afin de permettre les travaux curatifs SGRE.
    - "1007 - Remote Stop Owner" elle a été activée afin de permettre l'entretien des PPDMs.

Les arrêts "hub" " sont causés par
    - "3130 - pitch lubrification".

Les arrêts "Yaw" sont liés aux alarmes
    - "10100 - Yaw motor superhearted" et
    - "10401 - yaw fail stopped"

Les arrêts "Controller" sont déclanchés à cause du déchargement des UPS dans
les turbines.

Le MTBF le moins bon du mois est celui de la catégorie "Yaw" :  126 jours
'''

## Graph 6 ax9

In [33]:
df_ax9 = pd.merge(
        (results.groupby('StationId')
                       .agg(**{'Durée alarmes': ('Duration 115(s)', lambda x: x.sum()/3600),
                               'Durée autres': ('Duration 20-25(s)', lambda x: x.sum()/3600)})
                       .sort_values('Durée alarmes', ascending=False)
                 # .head(25)
                 .reset_index()
         ),
         (alarms.groupby('StationId')
                      .agg(**{'Freq alarmes': ('Alarmcode', lambda x: int(x[x == 115].count()/2)),
                              'Freq autres': ('Alarmcode', lambda x: x[x == 20].count())})
                      .reset_index()
         ),
         on='StationId'
        ).sort_values('Durée alarmes', ascending=False).head(20)


df_ax9['Duration_20_25 reel'] = df_ax9['Durée autres']
df_ax9['Durée autres'] = df_ax9['Duration_20_25 reel'] - df_ax9['Durée alarmes']
# df_ax9.plot(kind='bar', x='StationId', ax=ax9)

df_ax9

,StationId,Durée alarmes,Durée autres,Freq alarmes,Freq autres,Duration_20_25 reel
0,83,263.856111,0.151389,4,4,264.007500
1,7,118.671944,11.066667,30,30,129.738611
2,52,66.861667,4.476389,24,12,71.338056
3,66,64.722222,16.620000,23,19,81.342222
4,39,61.663333,11.924167,48,54,73.587500
5,92,57.570000,16.106111,40,20,73.676111
6,49,53.186389,9.734167,15,19,62.920556
7,19,47.688889,4.085278,19,17,51.774167
8,113,44.790278,11.235833,32,19,56.026111
9,118,44.623333,6.630833,12,13,51.254167


## Graphe 7 ax10

In [34]:
df_ax10 = (results[['StationId','ELNX', 'EL_indefini_left']]
                .groupby('StationId')
                .sum()
                .sort_values('ELNX', ascending=False)
                .head(20)
                .reset_index())

df_ax10 = round(df_ax10, 2).abs().head(20)

df_ax10

,StationId,ELNX,EL_indefini_left
0,7.0,250480.47,0.00
1,39.0,123493.82,73.87
2,66.0,116291.68,161.48
3,49.0,114044.76,0.00
4,52.0,90080.53,0.00
5,19.0,88520.44,0.00
6,6.0,79491.49,0.00
7,83.0,69586.15,0.00
8,97.0,69541.15,454.77
9,98.0,66180.04,278.66


## table 1 ax1 row 2

In [35]:

df_ax1 = pd.DataFrame(columns=['LTA-Lost Time', 'Indispo. Total %','Indispo. Tarec %', 'Indispo. Siemens %', 'Indispo. ONEE %', 'Indispo. Ebop %',
                               'Pertes élctriques en MWh', 'Power Boost en MWh', 'Performance moyenne des turbines',
                               'MTBF - Mean Time Between Failure', 'MTTR - Mean Time To Repair', 'MTTI - Mean Time To Intervention',
                               'Compteurs ONEE MWh'],
                     index=['Indicateurs annuels :', f'Indicateurs du mois {period} :'])
df_ax1 = df_ax1.fillna(0.)

In [36]:
df_ax1.iat[1, 1] = round(100*results['RealPeriod'].sum()/3600/24/131/31,2)
df_ax1.iat[1, 2] = round(100*results['Period 0(s)'].sum()/3600/24/131/31,2)
df_ax1.iat[1, 3] = round(100*results['Period 1(s)'].sum()/3600/24/131/31,2)

df_ax1.iat[1, 6] = round(results.wtc_ActPower_mean.sum()/6/1000 - onee_period,2)

df_ax1.iat[1, 7] = boost_period

#df_ax1.iat[1, 8] = ratio_period
#
#df_ax1.iat[1, 9]  = mtbf_period
#df_ax1.iat[1, 10] = mttr_period
#df_ax1.iat[1, 11] = mtti_period


# results.wtc_kWG1TotE_accum.sum() - results.wtc_kWG1Tot_accum.sum()
# results.wtc_ActPower_mean.sum()/6 - results.wtc_kWG1Tot_accum.sum()



In [37]:
df_ax1

,LTA-Lost Time,Indispo. Total %,Indispo. Tarec %,Indispo. Siemens %,Indispo. ONEE %,Indispo. Ebop %,Pertes élctriques en MWh,Power Boost en MWh,Performance moyenne des turbines,MTBF - Mean Time Between Failure,MTTR - Mean Time To Repair,MTTI - Mean Time To Intervention,Compteurs ONEE MWh
Indicateurs annuels :,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0
Indicateurs du mois 2020-08 :,0.0,1.76,0.33,1.43,0.0,0.0,4022.39,2290.329871,0.0,0.0,0.0,0.0,0.0


## table 2 ax2

In [27]:

df_ax2_header = ['', '', '', 'Travaux de maintenance']
df_ax2 = pd.DataFrame(columns=['Transformateur', 'Serial Defect', 'Corrosion', 'Inspection Fin de garantie',
                               'FSA', 'Assurance', 'Conformité DNSSI', 'Main Bearing', 'BAX', 'Darwin',
                               'Procédure de securité', 'SMI', 'Exploitation', 'Mesures', 'Audit ENGIE'],
                     index=['Etape terminée', 'Etape en cours'])

In [28]:
df_ax2

,Transformateur,Serial Defect,Corrosion,Inspection Fin de garantie,FSA,Assurance,Conformité DNSSI,Main Bearing,BAX,Darwin,Procédure de securité,SMI,Exploitation,Mesures,Audit ENGIE
Etape terminée,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Etape en cours,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Calcul Cumul

In [38]:
cumul_alarms = pd.DataFrame()
cumul_results = pd.DataFrame()

for month in range(1, period_month+1):

    month = str(month)

    alarms = read_sum(f'2020-{month.zfill(2)}')
    cumul_alarms = pd.concat([cumul_alarms, alarms])
    # -------------------------------------------------------------------------
    results = pd.read_pickle(f'../DATA/results/2020-{month.zfill(2)}.pkl')
    # results = results[['StationId', 'ELNX', 'Duration 115(s)', 'Duration 20-25(s)',
    #                    'Period 0(s)', 'Period 1(s)', 'RealPeriod',
    #                    'EL_indefini_left']]
    cumul_results = pd.concat([cumul_results, results])


DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2020-01-sum.mdb;
../DATA/SUM/2020-01-sum.mdb Loaded


C:\Users\Saad\Miniconda3\envs\dash-env\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2020-02-sum.mdb;
../DATA/SUM/2020-02-sum.mdb Loaded


C:\Users\Saad\Miniconda3\envs\dash-env\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2020-03-sum.mdb;
../DATA/SUM/2020-03-sum.mdb Loaded


...

C:\Users\Saad\Miniconda3\envs\dash-env\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2020-07-sum.mdb;
../DATA/SUM/2020-07-sum.mdb Loaded


C:\Users\Saad\Miniconda3\envs\dash-env\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=../DATA/SUM/2020-08-sum.mdb;
../DATA/SUM/2020-08-sum.mdb Loaded


C:\Users\Saad\Miniconda3\envs\dash-env\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [39]:
cumul_alarms.rename(columns={'StationNr': 'StationId'}, inplace=True)

# only run once otherwise negative turbine numbers
cumul_alarms['StationId'] = cumul_alarms['StationId'] - 2307404
cumul_results['StationId'] = cumul_results['StationId'] - 2307404

In [40]:
cumul_result_sum = pd.merge(cumul_alarms, error_list[[
                        'Alarmcode', 'Error Type', 'Error Group']],
                        on='Alarmcode',
                        how='inner', sort=False)

# Remove warnings
cumul_result_sum = cumul_result_sum.loc[cumul_result_sum['Error Type'].isin([1, 0])]

# apply cascade
cumul_alarms_result_sum = apply_cascade(cumul_result_sum)

#only keep  parent alarms
cumul_parent_result_sum = cumul_alarms_result_sum.query('TimeOn	 == NewTimeOn')
  
cumul_main_result_sum = cumul_alarms_result_sum.query('RealPeriod > @pd.Timedelta(0)')

## Table 2 ax1 row 1

In [41]:
nbr_jrs_total = len(pd.date_range(f'{period_year}-01-01', f'{lastday_period}', freq='D'))

df_ax1.iat[0, 1] = round(100*cumul_results['RealPeriod'].sum()/3600/24/131/nbr_jrs_total,2)
df_ax1.iat[0, 2] = round(100*cumul_results['Period 0(s)'].sum()/3600/24/131/nbr_jrs_total,2)
df_ax1.iat[0, 3] = round(100*cumul_results['Period 1(s)'].sum()/3600/24/131/nbr_jrs_total,2)

df_ax1.iat[0, 6] = round(cumul_results.wtc_ActPower_mean.sum()/6/1000 - last_onee_cumul,2)

df_ax1.iat[0, 7] = boost_cumul

# df_ax1.iat[0, 8] = ratio_cumul
# 
# df_ax1.iat[0, 9] =  mtbf_cumul
# df_ax1.iat[0, 10] = mttr_cumul
# df_ax1.iat[0, 11] = mtti_cumul
# df_ax1.iat[0, 12] = onee_cumul



In [45]:
df_ax1.to_clipboard()

## Graphe 1 ax3

In [48]:
df_ax3 = (cumul_main_result_sum.groupby('Error Group')
                .agg(Freq=('Alarmcode', 'count'),
                     Durée=('RealPeriod', lambda x: x.sum().total_seconds()/3600))
                .sort_values('Freq', ascending=False)
                .reindex(reorder)
                .dropna()
                .reset_index()
                )

# df_ax3.plot(kind='bar', x='Error Group', ax=ax3)
df_ax3

,Error Group,Freq,Durée
0,System,3596.0,7824.485225
1,Generator,34.0,49.965556
2,Hub,5214.0,735.846924
3,Gear,165.0,556.156978
4,Grid,452.0,391.821659
5,Rotor,59.0,113.099719
6,Hydraulics,1134.0,1466.043081
7,Environement,307.0,219.518615
8,Brake,397.0,223.334592
9,Yaw,5320.0,4153.840562


## Graphe 2 ax4

In [46]:
df_ax4

,Catégorie,MTTR,MTBF
0,Brake,10.135484,715.039227
1,Controller,11.783332,173.902778
2,Converter,12.033263,242.133397
3,Gear,11.341296,516.249668
4,Grid,9.004168,160.913844
5,Hub,11.511736,633.679435
6,Hydraulics,10.201698,142.667237
7,Miscellaneous,15.026806,13950.873883
8,Rotor,8.101708,1394.812429
9,System,9.449068,139.121289


## Graphe 5 ax8

In [38]:
df_ax8 = pd.merge(
        (cumul_results.groupby('StationId')
                       .agg(**{'Durée alarmes': ('Duration 115(s)', lambda x: x.sum()/3600),
                               'Durée autres': ('Duration 20-25(s)', lambda x: x.sum()/3600)})
                       .reset_index()
         ),
         (cumul_alarms.groupby('StationId')
                      .agg(**{'Freq alarmes': ('Alarmcode', lambda x: int(x[x == 115].count()/2)),
                              'Freq autres': ('Alarmcode', lambda x: x[x == 20].count())})
                      .reset_index()
         ),
        ).sort_values('Durée alarmes', ascending=False).head(20)

# df_ax8.plot(kind='bar', x='StationId', ax=ax8)

df_ax8['Duration_20_25 reel'] = df_ax8['Durée autres']
df_ax8['Durée autres'] = df_ax8['Duration_20_25 reel'] - df_ax8['Durée alarmes']

df_ax8

,StationId,Durée alarmes,Durée autres,Freq alarmes,Freq autres,Duration_20_25 reel
68,69,852.326667,320.897778,64,205,1173.224444
40,41,420.651944,344.513056,95,273,765.165000
33,34,415.475556,344.119167,129,272,759.594722
82,83,347.886389,399.963333,85,313,747.849722
65,66,341.645000,380.488889,173,310,722.133889
107,108,336.062222,495.928056,236,389,831.990278
100,101,330.964722,414.197222,149,346,745.161944
74,75,314.727500,372.348611,167,320,687.076111
128,129,303.789444,451.873889,91,370,755.663333
6,7,296.696944,346.343056,185,323,643.040000


## Graphe 7 ax18

In [39]:
df_ax18 = (cumul_results[['StationId','ELNX', 'EL_indefini_left']]
                .groupby('StationId')
                .sum()
                .sort_values('StationId')
                # .head(20)
                .reset_index())

df_ax18

,StationId,ELNX,EL_indefini_left
0,1,126789.384078,3535.84
1,2,140337.952267,922.79
2,3,138113.328691,1059.60
3,4,172929.627561,1355.02
4,5,138982.402890,2774.82
...,...,...,...
126,127,170582.449962,49790.61
127,128,28849.232719,24917.40
128,129,331780.795435,13916.89
129,130,50176.565918,22022.13


# Export Data to excel xlsxwriter

In [42]:
writer = pd.ExcelWriter(f'./input/{period}/output_xlsxwriter.xlsx')
# -------------------------------------------------------------------------------------------

workbook = writer.book
dashsheet = workbook.add_worksheet('Dash')

# -------------------------------------------------------------------------------------------

# MTBF_MTTR_cat_current_cumul.to_excel(writer, sheet_name='MTBF & MTTR cumul')

# -------------------------------------------------------------------------------------------

df_ax1.to_excel(writer, sheet_name='ax1')

# -------------------------------------------------------------------------------------------

df_ax3.to_excel(writer, index=False, sheet_name='ax3')

worksheet = writer.sheets['ax3']


def make_chart_ax3():
    column_chart = workbook.add_chart({'type': 'column'})

    column_chart.add_series({'name': ['ax3', 0, 2],
                             'categories': ['ax3', 1, 0, 13, 0],
                             'values': ['ax3', 1, 2, 13, 2], })

    line_chart = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart. We also set a
    # secondary Y axis via (y2_axis).
    line_chart.add_series({
        'values': ['ax3', 1, 1, 13, 1],
        'categories': ['ax3', 1, 0, 13, 0],
        'name': ['ax3', 0, 1],
        'y2_axis': True,
    })

    # Combine the charts.
    column_chart.combine(line_chart)

    # Configure the chart axes.
    # column_chart.set_x_axis({'name': df_ax3.columns[0]})
    column_chart.set_y_axis({'name': 'Durée en h'})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title({'name': 'Cumul annuel par type d\'alarme',
                            'name_font': {'size': 12, 'bold': True}})
    line_chart.set_y2_axis({'name': df_ax3.columns[1]})
    return column_chart


column_chart = make_chart_ax3()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax3()
# Insert the chart into the worksheet.
dashsheet.insert_chart('B2', column_chart)

# -------------------------------------------------------------------------------------------
# df_ax4.to_excel(writer, index=False, sheet_name='ax4')
# worksheet = writer.sheets['ax4']
# 
# 
# def make_chart_ax4():
#     column_chart = workbook.add_chart({'type': 'column'})
# 
#     column_chart.add_series({'name': ['ax4', 0, 2],
#                              'categories': ['ax4', 1, 0, 13, 0],
#                              'values': ['ax4', 1, 2, 13, 2]})
# 
#     line_chart = workbook.add_chart({'type': 'line'})
# 
#     # Configure the data series for the secondary chart. We also set a
#     # secondary Y axis via (y2_axis).
#     line_chart.add_series({
#         'values': ['ax4', 1, 1, 13, 1],
#         'categories': ['ax4', 1, 0, 13, 0],
#         'name': ['ax4', 0, 1],
#         'y2_axis': True,
#     })
# 
#     # Combine the charts.
#     column_chart.combine(line_chart)
#     # Configure the chart axes.
#     # column_chart.set_x_axis({'name': df_ax4.columns[0]})
#     column_chart.set_y_axis({'name': 'MTBF en j'})
#     column_chart.set_legend({'position': 'bottom'})
#     column_chart.set_title({'name': 'MTBF et MTTR par catégorie sur l\'année 2020',
#                             'name_font': {'size': 12, 'bold': True}})
#     line_chart.set_y2_axis({'name': 'MTTR en h'})
#     return column_chart
# 
# 
# column_chart = make_chart_ax4()
# # Insert the chart into the worksheet.
# worksheet.insert_chart('E2', column_chart)
# 
# column_chart = make_chart_ax4()
# # Insert the chart into the worksheet.
# dashsheet.insert_chart('J2', column_chart)

# -------------------------------------------------------------------------------------------
df_ax5.to_excel(writer, index=False, sheet_name='ax5')

worksheet = writer.sheets['ax5']


def make_chart_ax5():
    column_chart = workbook.add_chart({'type': 'column'})

    column_chart.add_series({'name': ['ax5', 0, 2],
                             'categories': ['ax5', 1, 0, 13, 0],
                             'values': ['ax5', 1, 2, 13, 2]})

    line_chart = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart. We also set a
    # secondary Y axis via (y2_axis).
    line_chart.add_series({
        'values': ['ax5', 1, 1, 13, 1],
        'categories': ['ax5', 1, 0, 13, 0],
        'name': ['ax5', 0, 1],
        'y2_axis': True,
    })

    # Combine the charts.
    column_chart.combine(line_chart)

    # Configure the chart axes.
    # column_chart.set_x_axis({'name': df_ax5.columns[0]})
    column_chart.set_y_axis({'name': 'Durée en h'})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title({'name': f'Type d\'alarme {period}',
                            'name_font': {'size': 12, 'bold': True}})

    line_chart.set_y2_axis({'name': df_ax5.columns[1]})
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax5()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax5()
# Insert the chart into the worksheet.
dashsheet.insert_chart('R2', column_chart)


# ------------------------------------------------------------------------------------------
df_ax6.to_excel(writer, index=False, sheet_name='ax6')

worksheet = writer.sheets['ax6']


def make_chart_ax6():
    column_chart = workbook.add_chart({'type': 'column'})

    column_chart.add_series({'name': ['ax6', 0, 2],
                             'categories': ['ax6', 1, 0, 20, 0],
                             'values': ['ax6', 1, 2, 20, 2]})

    line_chart = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart. We also set a
    # secondary Y axis via (y2_axis).
    line_chart.add_series({
        'values': ['ax6', 1, 1, 20, 1],
        'categories': ['ax6', 1, 0, 20, 0],
        'name': ['ax6', 0, 1],
        'y2_axis': True,
    })

    # Combine the charts.
    column_chart.combine(line_chart)

    # Configure the chart axes.
    column_chart.set_x_axis({'name': df_ax6.columns[0]})
    column_chart.set_y_axis({'name': 'Durée en h'})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title({'name': f'Alarmes {period}',
                            'name_font': {'size': 12, 'bold': True}})

    line_chart.set_y2_axis({'name': df_ax6.columns[1]})
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax6()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax6()
# Insert the chart into the worksheet.
dashsheet.insert_chart('Z2', column_chart)


# ------------------------------------------------------------------------------------------------
df_ax8.to_excel(writer, index=False, sheet_name='ax8')

worksheet = writer.sheets['ax8']


def make_chart_ax8():
    column_chart = workbook.add_chart({'type': 'column',
                                       'subtype': 'stacked'})

    column_chart.add_series({'name': ['ax8', 0, 1],
                             'categories': ['ax8', 1, 0, 20, 0],
                             'values': ['ax8', 1, 1, 20, 1]})

    column_chart.add_series({'name': ['ax8', 0, 2],
                             'categories': ['ax8', 1, 0, 20, 0],
                             'values': ['ax8', 1, 2, 20, 2]})

    line_chart = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart. We also set a
    # secondary Y axis via (y2_axis).
    line_chart.add_series({
        'values': ['ax8', 1, 3, 20, 3],
        'categories': ['ax8', 1, 0, 20, 0],
        'name': ['ax8', 0, 3],
        'y2_axis': True,
    })

    line_chart.add_series({
        'values': ['ax8', 1, 4, 20, 4],
        'categories': ['ax8', 1, 0, 20, 0],
        'name': ['ax8', 0, 4],
        'y2_axis': True,
    })

    # Combine the charts.
    column_chart.combine(line_chart)

    # Configure the chart axes.
    column_chart.set_x_axis({'name': df_ax8.columns[0]})
    column_chart.set_y_axis({'name': 'Durée en h'})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title({'name': 'Arrêts turbines : Cumul Annuel',
                            'name_font': {'size': 12, 'bold': True}})

    line_chart.set_y2_axis({'name': 'Freq'})
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax8()
# Insert the chart into the worksheet.
worksheet.insert_chart('G2', column_chart)

column_chart = make_chart_ax8()
# Insert the chart into the worksheet.
dashsheet.insert_chart('B17', column_chart)

# ----------------------------------------------------------------------------------------

df_ax9.to_excel(writer, index=False, sheet_name='ax9')

worksheet = writer.sheets['ax9']


def make_chart_ax9():
    column_chart = workbook.add_chart({'type': 'column',
                                       'subtype': 'stacked'})

    column_chart.add_series({'name': ['ax9', 0, 1],
                             'categories': ['ax9', 1, 0, 20, 0],
                             'values': ['ax9', 1, 1, 20, 1]})

    column_chart.add_series({'name': ['ax9', 0, 2],
                             'categories': ['ax9', 1, 0, 20, 0],
                             'values': ['ax9', 1, 2, 20, 2]})

    line_chart = workbook.add_chart({'type': 'line'})

    # Configure the data series for the secondary chart. We also set a
    # secondary Y axis via (y2_axis).
    line_chart.add_series({
        'values': ['ax9', 1, 3, 20, 3],
        'categories': ['ax9', 1, 0, 20, 0],
        'name': ['ax9', 0, 3],
        'y2_axis': True,
    })

    line_chart.add_series({
        'values': ['ax9', 1, 4, 20, 4],
        'categories': ['ax9', 1, 0, 20, 0],
        'name': ['ax9', 0, 4],
        'y2_axis': True,
    })

    # Combine the charts.
    column_chart.combine(line_chart)

    # Configure the chart axes.
    column_chart.set_x_axis({'name': df_ax9.columns[0]})
    column_chart.set_y_axis({'name': 'Durée en h'})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title({'name': f'Arrêts turbines {period}',
                            'name_font': {'size': 12, 'bold': True}})

    line_chart.set_y2_axis({'name': 'Freq'})
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax9()
# Insert the chart into the worksheet.
worksheet.insert_chart('G2', column_chart)

column_chart = make_chart_ax9()
# Insert the chart into the worksheet.
dashsheet.insert_chart('J17', column_chart)

# ----------------------------------------------------------------------------------------
df_ax10.to_excel(writer, index=False, sheet_name='ax10')

worksheet = writer.sheets['ax10']


def make_chart_ax10():
    column_chart = workbook.add_chart({'type': 'column',
                                       'subtype': 'stacked'})

    for col in range(1, 3):
        column_chart.add_series({'values': ['ax10', 1, col, 20, col],
                                 'categories': ['ax10', 1, 0, 20, 0],
                                 'name': ['ax10', 0, col]})

    # Configure the chart axes.
    column_chart.set_x_axis({'name': df_ax10.columns[0]})
    # column_chart.set_y_axis({'name': df_ax10.columns[1]})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title(
        {'name': f'Energie perdue selon FSA du {period}',
         'name_font': {'size': 12, 'bold': True}}
    )
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax10()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax10()
# Insert the chart into the worksheet.
dashsheet.insert_chart('R17', column_chart)

# ----------------------------------------------------------------------------------------
df_ax18.to_excel(writer, index=False, sheet_name='ax18')

worksheet = writer.sheets['ax18']


def make_chart_ax18():
    column_chart = workbook.add_chart({'type': 'column',
                                       'subtype': 'stacked'})

    for col in range(1, 3):
        column_chart.add_series({'values': ['ax18', 1, col, 131, col],
                                 'categories': ['ax18', 1, 0, 131, 0],
                                 'name': ['ax18', 0, col]})

    # Configure the chart axes.
    column_chart.set_x_axis({'name': df_ax18.columns[0]})
    # column_chart.set_y_axis({'name': df_ax18.columns[1]})
    column_chart.set_legend({'position': 'bottom'})
    column_chart.set_title(
        {'name': 'Energie perdue selon FSA cumulée sur l\'année 2020 en MWh',
         'name_font': {'size': 12, 'bold': True}}
    )
    return column_chart


# Insert the chart into the worksheet.
column_chart = make_chart_ax18()
# Insert the chart into the worksheet.
worksheet.insert_chart('E2', column_chart)

column_chart = make_chart_ax18()
# Insert the chart into the worksheet.
dashsheet.insert_chart('B32', column_chart, {'x_scale': 4, 'y_scale': 1})

# ----------------------------------------------------------------------------------------
# df_ax19.to_excel(writer, index=False, sheet_name='ax19')
# 
# worksheet = writer.sheets['ax19']
# 
# 
# def make_chart_ax19():
#     column_chart = workbook.add_chart({'type': 'column'})
# 
#     column_chart.add_series({'values': ['ax19', 1, 1, 131, 1],
#                              'categories': ['ax19', 1, 0, 131, 0],
#                              'name': ['ax19', 0, 1]})
# 
#     # Configure the chart axes.
#     column_chart.set_x_axis({'name': df_ax19.columns[0]})
#     # column_chart.set_y_axis({'name': df_ax19.columns[1]})
#     column_chart.set_legend({'position': 'bottom'})
#     column_chart.set_title(
#         {'name': f'Ratio Courbe Théorique vs Courbe Réelle du {period}',
#          'name_font': {'size': 12, 'bold': True}}
#     )
#     return column_chart
# 
# 
# # Insert the chart into the worksheet.
# column_chart = make_chart_ax19()
# # Insert the chart into the worksheet.
# worksheet.insert_chart('E2', column_chart)
# 
# column_chart = make_chart_ax19()
# # Insert the chart into the worksheet.
# dashsheet.insert_chart('B47', column_chart, {'x_scale': 4, 'y_scale': 1})


writer.save()

In [55]:
!jupyter nbconvert performance.ipynb --to markdown

[NbConvertApp] Converting notebook performance.ipynb to markdown
[NbConvertApp] Writing 77827 bytes to performance.md
